#Install necessary packages

In [1]:
!pip install nilearn==0.9.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 54.1 MB/s eta 0:00:00


#Import libraries

In [2]:
import os
import numpy as np
from pathlib import Path
from PIL import Image
import matplotlib
from matplotlib import pyplot as plt
from nilearn import datasets, plotting
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms

#Mount to drive

In [3]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
data_dir = '/content/drive/MyDrive/algonauts_2023_tutorial_data'
parent_submission_dir = '/content/drive/MyDrive/algonauts_2023_challenge_submission'

Mounted at /content/drive/


#Select device

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device(device)
print(device)

cpu


#Import data

In [5]:
subj = 1 #@param ["1", "2", "3", "4", "5", "6", "7", "8"] {type:"raw", allow-input: true}

In [6]:
class argObj:
  def __init__(self, data_dir, parent_submission_dir, subj):

    self.subj = format(subj, '02')
    self.data_dir = os.path.join(data_dir, 'subj'+self.subj)
    self.parent_submission_dir = parent_submission_dir
    self.subject_submission_dir = os.path.join(self.parent_submission_dir,
        'subj'+self.subj)

args = argObj(data_dir, parent_submission_dir, subj)

In [7]:
fmri_dir = os.path.join(args.data_dir, 'training_split', 'training_fmri')

rh_fmri = np.load(os.path.join(fmri_dir, 'rh_training_fmri.npy'))

print('\nRH training fMRI data shape:')
print(rh_fmri.shape)
print('(Training stimulus images × RH vertices)')

lh_fmri = np.load(os.path.join(fmri_dir, 'lh_training_fmri.npy'))

print('\nLH training fMRI data shape:')
print(lh_fmri.shape)
print('(Training stimulus images × LH vertices)')


RH training fMRI data shape:
(9841, 20544)
(Training stimulus images × RH vertices)

LH training fMRI data shape:
(9841, 19004)
(Training stimulus images × LH vertices)


In [8]:
train_img_dir  = os.path.join(args.data_dir, 'training_split', 'training_images')
test_img_dir  = os.path.join(args.data_dir, 'test_split', 'test_images')

# Create lists will all training and test image file names, sorted
train_img_list = os.listdir(train_img_dir)
train_img_list.sort()
test_img_list = os.listdir(test_img_dir)
test_img_list.sort()
print('Training images: ' + str(len(train_img_list)))
print('Test images: ' + str(len(test_img_list)))

Training images: 9841
Test images: 159


#Train Validation and Test Split

In [9]:
rand_seed = 5
np.random.seed(rand_seed)

# Calculate how many stimulus images correspond to 90% of the training data
num_train = int(np.round(len(train_img_list) / 100 * 90))
# Shuffle all training stimulus images
idxs = np.arange(len(train_img_list))
np.random.shuffle(idxs)
# Assign 90% of the shuffled stimulus images to the training partition,
# and 10% to the test partition
idxs_train, idxs_val = idxs[:num_train], idxs[num_train:]
# No need to shuffle or split the test stimulus images
idxs_test = np.arange(len(test_img_list))

print('Training stimulus images: ' + format(len(idxs_train)))
print('\nValidation stimulus images: ' + format(len(idxs_val)))
print('\nTest stimulus images: ' + format(len(idxs_test)))

Training stimulus images: 8857

Validation stimulus images: 984

Test stimulus images: 159


#Dataloader

In [10]:
# Define the custom dataset
class ImageDataset(Dataset):
    def __init__(self, imgs_paths, idxs, transform):
        self.imgs_paths = np.array(imgs_paths)[idxs]
        self.transform = transform

    def __len__(self):
        return len(self.imgs_paths)

    def __getitem__(self, idx):
        img_path = self.imgs_paths[idx]
        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img).to(device)
        return img

#Transfer Learning Model

In [11]:
# Define the transform for image preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # resize the images to 224x224 pixels
    transforms.ToTensor(),  # convert the images to a PyTorch tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # normalize the images color channels
])

model = models.resnet50(pretrained=True)
model.to(device)
model.eval()
feature_extractor = nn.Sequential(*list(model.children())[:-1])

class LinearizingEncodingModel(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim1, hidden_dim2, activation1, activation2,
                 bnorm1, bnorm2, dropout1, dropout_ratio1, dropout2, dropout_ratio2):
        super(LinearizingEncodingModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        if activation1:
            self.activation1 = activation1()
        if bnorm1:
            self.batchnorm1 = nn.BatchNorm1d(hidden_dim1)
        if dropout1:
            self.dropout1 = nn.Dropout(dropout_ratio1)
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        if activation2:
            self.activation2 = activation2()
        if bnorm2:
            self.batchnorm2 = nn.BatchNorm1d(hidden_dim2)
        if dropout2:
            self.dropout2 = nn.Dropout(dropout_ratio2)
        self.fc3 = nn.Linear(hidden_dim2, output_dim)

    def forward(self, x):
        x = self.fc1(x) #initial
        if hasattr(self, 'activation1'):
            x = self.activation1(x) #initial
        if hasattr(self, 'batchnorm1'):
            x = self.batchnorm1(x)
        if hasattr(self, 'dropout1'):
            x = self.dropout1(x)
        x = self.fc2(x) #initial
        if hasattr(self, 'activation2'):
            x = self.activation2(x)
        if hasattr(self, 'batchnorm2'):
            x = self.batchnorm2(x)
        if hasattr(self, 'dropout2'):
            x = self.dropout2(x)
        x = self.fc3(x)
        return x


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 102MB/s]


#Load Trained Models


In [15]:

lh_fmri_train = lh_fmri[idxs_train]
lh_fmri_val = lh_fmri[idxs_val]
rh_fmri_train = rh_fmri[idxs_train]
rh_fmri_val = rh_fmri[idxs_val]

# Get the output shape of the feature extractor layer
with torch.no_grad():
    sample_input = torch.zeros(1, 3, 224, 224).to(device)
    output = feature_extractor(sample_input)

#The hyperparameter values where selected using optuna
input_dim = output.shape[1]  # Set the dimensions for input and output of thr pretrained model
output_dim = rh_fmri_train.shape[1]
hidden_dim1 = 398
hidden_dim2 = 130
num_epochs = 50
activation1 = nn.Tanh
activation2 = nn.ReLU
bnorm1 = False
bnorm2 = False
dropout1 = True
dropout_ratio1 = 0.36525276151080455
dropout2 = False
dropout_ratio2 = 0.0
learning_rate = 0.006044678842679579
optimizer = optim.Adam
loss_function = nn.MSELoss()
batch_size = 150
weight_decay = 0.0007265659377076724

network_right = LinearizingEncodingModel(input_dim, output_dim, hidden_dim1,  hidden_dim2, activation1, activation2, bnorm1, bnorm2, dropout1, dropout_ratio1, dropout2, dropout_ratio2).to(device)
checkpoint = torch.load('/content/drive/MyDrive/resnet50_right_hemishpere.pt', map_location=device)
network_right.load_state_dict(checkpoint)

output_dim = lh_fmri_train.shape[1]
network_left = LinearizingEncodingModel(input_dim, output_dim, hidden_dim1,  hidden_dim2, activation1, activation2, bnorm1, bnorm2, dropout1, dropout_ratio1, dropout2, dropout_ratio2).to(device)
checkpoint = torch.load('/content/drive/MyDrive/resnet50_left_hemishpere.pt', map_location=device)
network_left.load_state_dict(checkpoint)

<All keys matched successfully>

#Make Predictions on Test Data

In [20]:
test_imgs_dataloader = DataLoader(
    ImageDataset(train_imgs_paths, idxs_test, transform),
    batch_size=batch_size
)

with torch.no_grad():
    test_features = []
    for data in test_imgs_dataloader:
        inputs = data.to(device)
        inputs = feature_extractor(inputs)
        inputs = inputs.view(inputs.size(0), -1)
        test_features.append(inputs)
    test_features = torch.cat(test_features, dim=0)

np.save("resnet50_test_features.npy", np.array(test_features))

In [21]:
from scipy.stats import pearsonr

lh_fmri_test = lh_fmri[idxs_test]
with torch.no_grad():
    network_left.eval()
    lh_fmri_test_pred = network_left(test_features)

lh_correlation = np.zeros(lh_fmri_test_pred.shape[1])
for v in range(lh_fmri_test_pred.shape[1]):
    lh_correlation[v] = pearsonr(lh_fmri_test_pred[:, v].detach().cpu().numpy(), lh_fmri_test[:, v])[0]

rh_fmri_test = rh_fmri[idxs_test]
with torch.no_grad():
    network_right.eval()
    rh_fmri_test_pred = network_right(test_features)

rh_correlation = np.zeros(rh_fmri_test_pred.shape[1])
for v in range(rh_fmri_test_pred.shape[1]):
    rh_correlation[v] = pearsonr(rh_fmri_test_pred[:, v].detach().cpu().numpy(), rh_fmri_test[:, v])[0]

array([0.14262287, 0.3032777 , 0.39484403, ..., 0.13386706, 0.21856263,
       0.16511646], dtype=float32)

#Visualization of the fMRI image responses of all vertices on a brain surface map

In [ ]:
img = 25 #@param
hemisphere = 'left'

if img < 0 or img > 158:
    print("Please select a number between 1 and 159")


# Load the image
img_dir = os.path.join(train_img_dir, train_img_list[idxs_test[img]])
train_img = Image.open(img_dir).convert('RGB')

# Plot the image


plt.axis('off')
plt.imshow(train_img)
plt.title('Test image: ' + str(img));

# Load the brain surface map of all vertices
roi_dir = os.path.join(args.data_dir, 'roi_masks',
    hemisphere[0]+'h.all-vertices_fsaverage_space.npy')
fsaverage_all_vertices = np.load(roi_dir)

# Map the fMRI data onto the brain surface map
fsaverage_response = np.zeros(len(fsaverage_all_vertices))
if hemisphere == 'left':
    fsaverage_response[np.where(fsaverage_all_vertices)[0]] = lh_fmri_test_pred[img]
elif hemisphere == 'right':
    fsaverage_response[np.where(fsaverage_all_vertices)[0]] = rh_fmri_test_pred[img]

# Create the interactive brain surface map
fsaverage = datasets.fetch_surf_fsaverage('fsaverage')
view = plotting.view_surf(
    surf_mesh=fsaverage['infl_'+hemisphere],
    surf_map=fsaverage_response,
    bg_map=fsaverage['sulc_'+hemisphere],
    threshold=1e-14,
    cmap='cold_hot',
    colorbar=True,
    title='All vertices, '+hemisphere+' hemisphere'
    )
view

#Visualization of the fMRI image responses of a chosen ROI on a brain surface map

In [ ]:
img = 25 #@param
hemisphere = 'left' #@param ['left', 'right'] {allow-input: true}
roi = "EBA" #@param ["V1v", "V1d", "V2v", "V2d", "V3v", "V3d", "hV4", "EBA", "FBA-1", "FBA-2", "mTL-bodies", "OFA", "FFA-1", "FFA-2", "mTL-faces", "aTL-faces", "OPA", "PPA", "RSC", "OWFA", "VWFA-1", "VWFA-2", "mfs-words", "mTL-words", "early", "midventral", "midlateral", "midparietal", "ventral", "lateral", "parietal"] {allow-input: true}

# Load the image
img_dir = os.path.join(train_img_dir, train_img_list[idxs_test[img]])
train_img = Image.open(img_dir).convert('RGB')

# Plot the image
plt.figure()
plt.axis('off')
plt.imshow(train_img)
plt.title('Test image: ' + str(img));

# Define the ROI class based on the selected ROI
if roi in ["V1v", "V1d", "V2v", "V2d", "V3v", "V3d", "hV4"]:
    roi_class = 'prf-visualrois'
elif roi in ["EBA", "FBA-1", "FBA-2", "mTL-bodies"]:
    roi_class = 'floc-bodies'
elif roi in ["OFA", "FFA-1", "FFA-2", "mTL-faces", "aTL-faces"]:
    roi_class = 'floc-faces'
elif roi in ["OPA", "PPA", "RSC"]:
    roi_class = 'floc-places'
elif roi in ["OWFA", "VWFA-1", "VWFA-2", "mfs-words", "mTL-words"]:
    roi_class = 'floc-words'
elif roi in ["early", "midventral", "midlateral", "midparietal", "ventral", "lateral", "parietal"]:
    roi_class = 'streams'

# Load the ROI brain surface maps
challenge_roi_class_dir = os.path.join(args.data_dir, 'roi_masks',
    hemisphere[0]+'h.'+roi_class+'_challenge_space.npy')
fsaverage_roi_class_dir = os.path.join(args.data_dir, 'roi_masks',
    hemisphere[0]+'h.'+roi_class+'_fsaverage_space.npy')
roi_map_dir = os.path.join(args.data_dir, 'roi_masks',
    'mapping_'+roi_class+'.npy')
challenge_roi_class = np.load(challenge_roi_class_dir)
fsaverage_roi_class = np.load(fsaverage_roi_class_dir)
roi_map = np.load(roi_map_dir, allow_pickle=True).item()

# Select the vertices corresponding to the ROI of interest
roi_mapping = list(roi_map.keys())[list(roi_map.values()).index(roi)]
challenge_roi = np.asarray(challenge_roi_class == roi_mapping, dtype=int)
fsaverage_roi = np.asarray(fsaverage_roi_class == roi_mapping, dtype=int)

# Map the fMRI data onto the brain surface map
fsaverage_response = np.zeros(len(fsaverage_roi))
if hemisphere == 'left':
    fsaverage_response[np.where(fsaverage_roi)[0]] = \
        lh_fmri_test_pred.cpu().detach().numpy()[img,np.where(challenge_roi)[0]]
elif hemisphere == 'right':
    fsaverage_response[np.where(fsaverage_roi)[0]] = \
        rh_fmri_test_pred.cpu().detach().numpy()[img,np.where(challenge_roi)[0]]

# Create the interactive brain surface map
fsaverage = datasets.fetch_surf_fsaverage('fsaverage')
view = plotting.view_surf(
    surf_mesh=fsaverage['infl_'+hemisphere],
    surf_map=fsaverage_response,
    bg_map=fsaverage['sulc_'+hemisphere],
    threshold=1e-14,
    cmap='cold_hot',
    colorbar=True,
    title=roi+', '+hemisphere+' hemisphere'
    )
view